# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_csv = pd.read_csv('../output_data/cities.csv')
cities_csv.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Country
0,Rikitea,-23.12,-134.97,73.67,80,0,17.07,1603111965,PF
1,Bluff,-46.60,168.33,46.99,85,96,18.01,1603111965,NZ
2,Esperance,-33.87,121.90,60.33,84,0,15.12,1603111965,AU
3,Shouguang,36.88,118.74,61.00,50,0,7.05,1603111965,CN
4,Khorixas,-20.37,14.97,103.03,7,0,3.42,1603111966,NaN


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = cities_csv[["Lat", "Lng"]].astype(float)
max_humdity = cities_csv['Humidity'].max().astype(float)


humidity_weight = cities_csv['Humidity'].astype(float)

# create figure
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity_weight, 
                                 dissipating=False, max_intensity=max_humdity, 
                                 point_radius=2)
fig.add_layer(heat_layer)


fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# A max temperature lower than 70 degrees but higher than 60.
# Wind speed less than 5 mph.
# More than 10% cloudiness.


ideal_weather_df = cities_csv.copy()
ideal_weather_df = ideal_weather_df[(ideal_weather_df['Max Temp'] < 70) & (ideal_weather_df['Max Temp'] > 60)]
ideal_weather_df = ideal_weather_df[ideal_weather_df["Wind Speed"] <= 5]
ideal_weather_df = ideal_weather_df[ideal_weather_df["Cloudiness"] >= 10]

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = ideal_weather_df.copy()
hotel_df['Hotel Name'] = ""
print(hotel_df.City.count())
hotel_df.head()

26


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Country,Hotel Name
17,Flinders,-34.58,150.86,62.01,78,100,1.01,1603111969,AU,
28,Geraldton,-28.77,114.60,66.99,76,87,4.00,1603111971,AU,
119,Gorodovikovsk,46.09,41.93,60.94,91,100,3.83,1603112112,RU,
126,Sakaiminato,35.55,133.23,62.92,52,96,2.80,1603112113,JP,
142,Khonsa,27.02,95.57,66.85,96,70,4.41,1603112176,IN,


In [7]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# happy testing small list of happy
hotel_df_mini = hotel_df.iloc[0:10]

error_number = 0 

for index, row in hotel_df.iterrows():
    
    coords = f"{hotel_df.Lat[index]},{hotel_df.Lng[index]}"
    city = hotel_df.City[index]

    params = {
        "location": coords, 
        "radius": "5000",
        "type": "lodging", 
        "key": g_key,
    }

    response = requests.get(base_url, params=params).json()
    try: 
        hotel_df.loc[index, "Hotel Name"] = response['results'][1]['name']
    except(KeyError, IndexError):
        error_number += 1


print(f"There were {error_number} errors")

There were 3 errors


In [8]:
hotel_df['Hotel Name'].replace("", np.nan, inplace=True)
hotel_df.dropna(how="any", inplace=True)

In [9]:
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Country,Hotel Name
17,Flinders,-34.58,150.86,62.01,78,100,1.01,1603111969,AU,Shellharbour Beachside Holiday Park
28,Geraldton,-28.77,114.60,66.99,76,87,4.00,1603111971,AU,Hospitality Geraldton
126,Sakaiminato,35.55,133.23,62.92,52,96,2.80,1603112113,JP,Italian pasley
237,Wanxian,30.80,108.39,63.57,78,100,2.71,1603112318,CN,Kinglead Hotel
263,Camacha,33.08,-16.33,67.57,77,100,3.80,1603112324,PT,Hotel Praia Dourada


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Display figure

locations = cities_csv[["Lat", "Lng"]].astype(float)
max_humdity = cities_csv['Humidity'].max().astype(float)
humidity_weight = cities_csv['Humidity'].astype(float)

# create figure
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity_weight, 
                                 dissipating=False, max_intensity=max_humdity, 
                                 point_radius=2)
fig.add_layer(heat_layer)
fig.add_layer(markers)
fig


Figure(layout=FigureLayout(height='420px'))